# 1. chatGPT API 사용법

이번 챕터에서는 chatGPT API 사용법을 익혀봅니다.

## openai client library

python으로 chatGPT API를 사용하려면 openai에서 제공하는 client library를 사용하는 것이 편합니다.  
설치한 다음, import 해주고 앞서 발급받은 API key를 지정해주겠습니다.

In [1]:
!pip install openai


[notice] A new release of pip is available: 23.1.2 -> 23.3.2
[notice] To update, run: pip install --upgrade pip


In [5]:
import os

In [6]:
from openai import OpenAI

client = OpenAI(
    # This is the default and can be omitted
    api_key=os.environ.get("OPENAI_API_KEY"),
)

## ChatCompletion API
openai 라이브러리를 통해서 openai가 제공하는 다양한 API들을 사용할 수 있습니다. 그 중에서 우리가 앞으로 가장 많이 쓰게 될 API는 ChatCompletion입니다. 이는 마치 채팅을 나누는 것 처럼, 이전 대화 텍스트를 전달하면 이어지는 대화를 생성해주는 API입니다. 

### 기본 사용법
ChatCompletion API는 model과 messages 두 파라미터를 전달해서 사용할 수 있습니다. 응답은 json 형태로 리턴되며, 이를 잘 파싱해서 생성된 텍스트를 읽어올 수 있습니다.

In [22]:
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "user", "content": "안녕하세요!"}
    ]
)

In [23]:
response

ChatCompletion(id='chatcmpl-8aI7x4C4o5eqpebUcxSrzPe8BAZ24', choices=[Choice(finish_reason='stop', index=0, message=ChatCompletionMessage(content='안녕하세요! 반갑습니다. 무엇을 도와드릴까요?', role='assistant', function_call=None, tool_calls=None), logprobs=None)], created=1703660181, model='gpt-3.5-turbo-0613', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=26, prompt_tokens=13, total_tokens=39))

### 텍스트 생성 결과 파싱

텍스트 생성 결과는 JSON 형태로 리턴됩니다. 다른 정보들도 같이 잔뜩 리턴되었는데, 그 중에서 텍스트만 읽어오겠습니다.

In [24]:
def parse_response(response):
    return  response.choices[0].message.content

In [26]:
print(parse_response(response))

안녕하세요! 반갑습니다. 무엇을 도와드릴까요?


## model 파라미터 지정
model은 대화 생성에 어떤 모델을 사용할 지 결정하는 파라미터입니다. 대표적으로 "gpt-3.5-turbo"와 "gpt-4-1106-preview"를 지정할 수 있습니다.

gpt-3.5-turbo는 성능이 준수하고 가격이 저렴합니다. 때문에 앞으로 우리는 대부분 이 모델을 이용할 예정입니다. 다만, 복잡한 테스크의 경우에는 성능이 떨어집니다. 반면에 gpt-4-1106-preview는 기가막힌 텍스트를 생성하는 대신, 10배 정도 더 비쌉니다.

한번 삼행시를 지어달라는 복잡한 테스크로 두 모델의 성능을 비교해보겠습니다.

### gpt-3.5-turbo로 복잡한 테스크 수행

In [29]:
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "user", "content": "지피티로 삼행시를 지어줘!"}
    ]
)
print(parse_response(response))

지금 나와 음성 AI,
함께 삼행시를 만들래.
문장 세 줄로 이야기,
함께 즐기자 백세살이.


### gpt-4로 복잡한 테스크 수행

In [30]:
response = client.chat.completions.create(
        model="gpt-4-1106-preview",
        messages=[
            {"role": "user", "content": "지피티로 삼행시를 지어줘!"}
        ]
    )
print(parse_response(response))

물론이죠. 제가 GPT(Global Pre-trained Transformer)를 말씀하시는 거라면 아래와 같이 삼행시를 지어 보겠습니다:

**지**금 여기서
**피**어나는 인공지능의
**티**끌 모아 태산처럼 지식을 전해줄게요!


힘의 차이가 느껴지시나요? 이처럼 텍스트 생성 자체는 gpt-4가 압도적입니다. 

### 어떤 모델을 선택하는 것이 좋을까?

gpt-3.5-turbo는 어지간한 테스크에 대해서 준수한 성능을 보여줍니다. 때문에 gpt-3.5-turbo 만으로도 내가 원하는 수준의 텍스트가 생성되는지 먼저 확인해보는 것이 좋습니다. 만약 결과가 불만족스럽고, 비용을 지불해서라도 퀄리티를 높여야한다면 gpt-4-1106-preview를 선택하면 됩니다. (참고로 펭추리는 3.5와 4를 적절히 섞어서 사용합니다 ㅎㅎ)

이 외에도 모델이 학습된 시점이나 입력으로 받을 수 있는 최대 토큰 수에 따라서 다양한 종류의 모델들이 있습니다. 궁금하신 분들은 아래 링크로 이동해서 모델마다 어떤 차이가 있는지 살펴보세요.

https://platform.openai.com/docs/models/gpt-4
https://platform.openai.com/docs/models/gpt-3-5

## role 파라미터
그 다음 messages 파라미터를 지정해주어야 합니다. 여기에는 유저와 AI가 나눈 대화를 리스트 형태로 전달해주어야 합니다. 이 때, 대화의 화자와 내용을 딕셔너리 형태로 전달해주어야 합니다. 

```python
{"role": "화자", "content": "대화내용"}
```

role에는 user, assistant, system이 있습니다. user는 유저, assistant는 chatGPT를 가리킵니다. system은 AI에게 역할을 부여할 때 사용하는데, 잠시 뒤에 알아보겠습니다. 이제 messages 파라미터를 전달할 때, 유저가 입력한 텍스트와 AI가 생성한 텍스트를 구분지어서 전달할 수 있습니다.

### AI와 대화나누기
한번 user와 assistant가 이전에 나눈 대화를 파라미터로 전달하고, 대화를 이어서 생성해보겠습니다.

In [31]:
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "user", "content": "내일 점심은 뭘 먹는게 좋을까?"},
    ]
)
print(parse_response(response))

뭘 좋아하시나요?


In [32]:
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "user", "content": "내일 점심은 뭘 먹는게 좋을까?"},
        {"role": "assistant", "content": "뭘 좋아하시나요?"},
        {"role": "user", "content": "가벼운 음식이면 좋겠어!"},
    ]
)
print(parse_response(response))

가벼운 음식 추천해드릴게요! 

1. 샐러드: 신선한 채소와 고기나 해산물을 섞어 만든 샐러드는 가벼우면서도 영양가가 높아요. 드레싱은 식초나 레몬 주스를 활용하여 칼로리를 줄일 수도 있어요.

2. 샌드위치: 다양한 재료를 사이에 넣어 만든 샌드위치는 가벼우면서도 포만감을 줄 수 있어요. 채소와 닭가슴살이나 햄, 치즈 등을 넣어 맛을 더할 수 있어요.

3. 라면: 라면도 가볍게 먹을 수 있는 메뉴 중 하나에요. 면만 먹지 말고 채소나 달걀, 김치 등을 추가하여 영양도 챙기는 것이 좋아요.

4. 국수: 면을 활용한 음식인 국수도 가벼운 선택 중 하나에요. 소고기 육수나 채소 스프로 만들어져 영양도 함께 챙길 수 있어요.

5. 전: 부침개인 전은 가벼우면서도 다양한 종류와 맛을 즐길 수 있는 메뉴에요. 계란이나 양파, 김치, 야채 등을 활용하여 만들 수 있어요.

위의 음식 중 원하시는 것을 선택하셔서 가볍게 즐기시면 좋을 것 같아요!


### AI와 대화하기 함수화

In [40]:
memory = []
while True:
    user_input = input("입력:")
    if user_input == "q":
        break
    memory.append({"role": "user", "content": user_input})
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=memory
    )
    generated_message = parse_response(response)
    print(generated_message)
    memory.append({"role": "assistant", "content": generated_message})

입력: 안녕?


안녕하세요! 무엇을 도와드릴까요?


입력: 점심 메뉴 추천 좀


물어보셔서 감사합니다! 어떤 종류의 음식을 선호하시나요?


입력: 한식이지 ㅎㅎ


한식 메뉴를 추천해드릴게요! 여러 가지 옵션 중에서 선택하실 수 있습니다:

1. 비빔밥: 맛있고 영양가도 높은 전통적인 한식입니다. 채소, 고기, 계란 등을 섞어 먹는 방식으로, 다양한 버전을 즐길 수 있어요.

2. 불고기: 부드럽고 달콤한 소고기를 양념하여 구워낸 요리로, 밥과 국, 반찬과 함께 즐기면 좋아요.

3. 제육볶음: 돼지고기와 야채를 고춧가루와 간장 등으로 볶아낸 것으로, 매콤한 맛이 일품입니다.

4. 김치찌개: 김치와 돼지고기, 된장, 물 등으로 만든 국물 요리로, 따뜻하고 짭짤한 맛이 느껴집니다.

5. 삼계탕: 닭고기와 쌀, 약초 등으로 쑥을 넣어 끓여 만든 건강한 한식요리입니다.

이런 한식 메뉴 중 어떤 것을 드시고 싶으신가요?


입력: 세번째 메뉴 설명해줘


제육볶음은 한국의 대표적인 볶음요리 중 하나입니다. 돼지고기를 네모난 조각으로 잘라서 간장, 고춧가루, 설탕, 다진 마늘 등으로 양념한 후 중간 불에서 볶는 방법으로 만들어집니다.

양념된 돼지고기를 먼저 볶고, 채소인 양파, 당근, 대파, 얼갈이배추 등을 함께 넣어 볶아냅니다. 양념과 채소의 조화로 매콤하고 달콤한 맛을 내면서도 야채의 신선한 풍미가 느껴집니다.

선택적으로 고춧가루나 고추장 등으로 맵기를 조절할 수 있으며, 볶음 과정에서 추가적으로 간을 맞출 수도 있습니다. 

제육볶음은 밥과 함께 먹을 수도 있고, 김밥이나 떡볶이 등 다양한 요리에 넣어 맛을 즐길 수 있습니다. 매콤하고 짭짤한 맛으로 여러분의 입맛을 돋워주는 한식요리입니다.


입력: q


### AI에게 역할 부여하기

대화를 시작할 때, role을 system으로 지정한 다음, AI에게 역할과 성격을 부여할 수 있습니다. 만약 이런 system 메세지를 설정하지 않는다면 chatGPT는 기본적으로 "You are an helpful assistant."라는 역할이 부여되게 됩니다.

```python
messages = [
    {"role": "system", "content": "당신은 주식 분석 전문가입니다."},
    ...
]
```

한번 내일 점심 메뉴를 물어보는 대화에 system message를 추가해보겠습니다.

In [34]:
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": "당신은 전라도 사투리를 구사하는 아저씨입니다."},
        {"role": "user", "content": "내일 점심은 뭘 먹는게 좋을까요?"},
    ]
)
print(parse_response(response))

아이고, 내일 점심은 장어구이 어떠냐? 김칫국도 함께 먹으면 제격이야. 김치와 반찬 별미 따로 준비해두면 좋을 걸? 그러니까 김치와 함께 먹을 김밥이나 배추김치를 차려두는 건 어떠냐고? 맛있게 먹으면 내가 더 좋아하겠네!


## Streaming

지금까지는 모든 텍스트가 생성되면 리턴 받는 식으로 요청을 보냈습니다. 그런데 우리가 chatGPT를 써보면, 곧바로 답변을 타닥타닥 생성합니다. 

API로도 이 기능을 사용할 수 있습니다. 먼저 요청을 보낼 때 stream 옵션을 True로 설정해줍니다. 그 다음, for문을 이용해서 응답으로부터 생성된 텍스트를 쭉쭉 읽어오는 겁니다.

In [39]:
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "user", "content": "내일 점심은 뭘 먹을까?"},
    ],
    stream=True
)
for chunk in response:
    delta = chunk.choices[0].delta
    if delta.content:
        print(delta.content, end="")

제가 추천하는 메뉴는 무엇이든지 좋습니다! 하지만 여러분의 선호도나 식사 습관에 따라 다를 수 있으니까요. 어떤 음식을 좋아하시나요? 추가 정보를 알려주시면 더 구체적인 추천을 할 수 있을 것 같아요.

유저가 질문하자마자 답변이 생성되는 걸 보여줄 수 있어서, 훨씬 유저 경험이 좋습니다. 뒤에 이어지는 서비스 개발 예시들에서도 유용하게 사용되니 잘 기억해주세요.|

## 마치며

지금까지 chatGPT API의 거의 모든 사용법을 알아봤습니다. 생각보다 되게 쉽죠? 이 정도만 빠삭하게 알고 있어도 정말 다양한 LLM을 이용한 서비스들을 만들 수 있습니다. 다음 강의부터 본격적으로 재밌는 프로젝트들을 직접 만들어 보겠습니다.